In [ ]:
import fiona
import geopandas as gpd

In [ ]:
# Pad naar je bestand
# Download bestuurlijke grenzen van PDOK.nl
path = r"..\data\bestuurlijkegrenzen.gpkg".replace("\\", "/")

In [ ]:
# Bekijk alle lagen
layers = fiona.listlayers(path)
print(layers)  # ['gemeenten', 'provincies', 'landsgrens']

In [ ]:
# import gemeentegebied alkmaar
gdf = gpd.read_file(path, layer="gemeenten")
alkmaar = gdf[gdf["gemeentenaam"] == "Alkmaar"]

# Controleer CRS
print(alkmaar.crs)

In [ ]:

# Zet om naar WGS84 (lon/lat)
alkmaar_WGS84 = alkmaar.to_crs(epsg=4326)

# Kies de eerste polygon als deze een MultiPolygon is
if alkmaar_WGS84.geometry.iloc[0].geom_type == "MultiPolygon":
    alkmaar_WGS84.geometry = alkmaar_WGS84.geometry.apply(
        lambda geom: geom.geoms[0] if geom.geom_type == "MultiPolygon" else geom
    )

# Opslaan als nieuwe, enkelvoudige polygon
alkmaar_WGS84.to_file("../data/alkmaar_single_polygon.geojson", driver="GeoJSON")